In [1]:
%pip install backports.tarfile
%pip install zss

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Procesamiento de los datasets

In [2]:
import os
from collections import OrderedDict

base_path = 'IR-Plag-Dataset'
pairs_dict = OrderedDict()

for case in sorted(os.listdir(base_path)):
    case_path = os.path.join(base_path, case)
    if not os.path.isdir(case_path):
        continue

    dataset_name = 'IR-Plag'
    orig_path = os.path.join(case_path, 'original')
    orig_file = next(os.scandir(orig_path)).path
    orig_id   = f'{case}-ORIG'

    nonplag_path = os.path.join(case_path, 'non-plagiarized')
    if os.path.exists(nonplag_path):
        for folder in sorted(os.listdir(nonplag_path)):
            folder_path = os.path.join(nonplag_path, folder)
            if not os.path.isdir(folder_path):
                continue
            file_path = next(os.scandir(folder_path)).path
            file_id   = f'{case}-NP-{folder}'
            pair_id   = f'{orig_id}_{file_id}'
            pairs_dict[pair_id] = {
                'idcode1': orig_id,
                'code1'  : orig_file,
                'idcode2': file_id,
                'code2'  : file_path,
                'result' : 0,
                'dataset': dataset_name
            }
            
            inverted_pair_id = f'{file_id}_{orig_id}'
            pairs_dict[inverted_pair_id] = {
                'idcode1': file_id,
                'code1'  : file_path,
                'idcode2': orig_id,
                'code2'  : orig_file,
                'result' : 0,
                'dataset': dataset_name
            }

    plag_path = os.path.join(case_path, 'plagiarized')
    if os.path.exists(plag_path):
        for level in sorted(os.listdir(plag_path)):
            level_path = os.path.join(plag_path, level)
            for folder in sorted(os.listdir(level_path)):
                folder_path = os.path.join(level_path, folder)
                if not os.path.isdir(folder_path):
                    continue
                file_path = next(os.scandir(folder_path)).path
                file_id   = f'{case}-{level}-{folder}'
                pair_id   = f'{orig_id}_{file_id}'
                pairs_dict[pair_id] = {
                    'idcode1': orig_id,
                    'code1'  : orig_file,
                    'idcode2': file_id,
                    'code2'  : file_path,
                    'result' : 1,
                    'dataset': dataset_name
                }
                
                inverted_pair_id = f'{file_id}_{orig_id}'
                pairs_dict[inverted_pair_id] = {
                    'idcode1': file_id,
                    'code1'  : file_path,
                    'idcode2': orig_id,
                    'code2'  : orig_file,
                    'result' : 1,
                    'dataset': dataset_name
                }

print(len(pairs_dict))


920


In [ ]:
import os
import pandas as pd

conplag_base     = 'conplag_version_2/versions'
conplag_code_dir = os.path.join(conplag_base, 'version_2')
conplag_labels   = os.path.join(conplag_base, 'labels.csv')

df = pd.read_csv(conplag_labels)

for _, row in df.iterrows():
    sub1, sub2, verdict = row['sub1'], row['sub2'], int(row['verdict'])

    codeid1  = str(sub1)
    codeid2  = str(sub2)
    pair_id  = f'{codeid1}_{codeid2}'

    folder_path = os.path.join(conplag_code_dir, pair_id)
    file1_path  = os.path.join(folder_path, f'{codeid1}.java')
    file2_path  = os.path.join(folder_path, f'{codeid2}.java')

    if not (os.path.exists(file1_path) and os.path.exists(file2_path)):
        continue

    pairs_dict[pair_id] = {
        'idcode1': codeid1,
        'code1'  : file1_path,          
        'idcode2': codeid2,
        'code2'  : file2_path,          
        'result' : verdict,             
        'dataset': 'conplag_version_2'
    }
    
    inverted_pair_id = f'{codeid2}_{codeid1}'
    pairs_dict[inverted_pair_id] = {
        'idcode1': codeid2,
        'code1'  : file2_path,          
        'idcode2': codeid1,
        'code2'  : file1_path,          
        'result' : verdict,             
        'dataset': 'conplag_version_2'
    }

print(f' Total de pares tras añadir CONPLAG: {len(pairs_dict)}')

✅ Total de pares tras añadir CONPLAG: 2742


In [ ]:
import os, random

base_path   = 'fire14-source-code-training-dataset'
java_path   = os.path.join(base_path, 'java')
qrel_path   = os.path.join(base_path, 'SOCO14-java.qrel')
dataset_name = 'FIRE14'

all_files = [f for f in os.listdir(java_path) if f.endswith('.java')]
file_set  = set(all_files)

positive_pairs = set()
with open(qrel_path, encoding='utf-8') as fh:
    for line in fh:
        f1, f2 = line.split()[:2]
        if not f1.endswith('.java'): f1 += '.java'
        if not f2.endswith('.java'): f2 += '.java'
        if f1 in file_set and f2 in file_set:
            positive_pairs.add((f1, f2))

positive_ids = list({x for p in positive_pairs for x in p})
negatives = set()
while len(negatives) < len(positive_pairs):
    f1, f2 = random.sample(positive_ids, 2)
    if f1 != f2 and (f1, f2) not in positive_pairs and (f2, f1) not in positive_pairs:
        negatives.add((f1, f2))

def add_pair(f1, f2, label):
    p1, p2 = os.path.join(java_path, f1), os.path.join(java_path, f2)
    if not (os.path.exists(p1) and os.path.exists(p2)): return
    pair_id = f'{f1}_{f2}'
    pairs_dict[pair_id] = {
        'idcode1': f1,
        'code1'  : p1,
        'idcode2': f2,
        'code2'  : p2,
        'result' : label,
        'dataset': dataset_name
    }
    
    inverted_pair_id = f'{f2}_{f1}'
    pairs_dict[inverted_pair_id] = {
        'idcode1': f2,
        'code1'  : p2,
        'idcode2': f1,
        'code2'  : p1,
        'result' : label,
        'dataset': dataset_name
    }

for f1, f2 in positive_pairs: add_pair(f1, f2, 1)
for f1, f2 in negatives:      add_pair(f1, f2, 0)

print(f' Total de pares tras añadir FIRE14: {len(pairs_dict)}')


✅ Total de pares tras añadir FIRE14: 3078


# Calculo de features

In [5]:
def read_code(file_path):
    with open(file_path, encoding='utf-8') as fh:
        return fh.read()

In [ ]:
from pathlib import Path
import re, javalang
from javalang import tree as jt 
import numpy as np
from simhash import Simhash
from collections import Counter
from tqdm import tqdm

_token_pat = re.compile(r'[A-Za-z_]\w+|\d+|==|!=|<=|>=|&&|\|\||[^\sA-Za-z0-9_]')

# Funciones para calcular indices de similitud
def clean_code(src:str)->str:
    
    src = re.sub(r'//.*?$|/\*.*?\*/', '', src, flags=re.S|re.M)  # comentarios
    return '\n'.join(line.strip() for line in src.splitlines() if line.strip())

def text_features(src:str):
    
    tokens = _token_pat.findall(clean_code(src))
    freq   = Counter(tokens)
    for tok, w in freq.items():
        yield tok, w                    

def simhash_text(src:str)->Simhash:
    return Simhash(list(text_features(src)))

def text_similarity(code_a:str, code_b:str)->float:
    h1, h2 = simhash_text(code_a), simhash_text(code_b)
    return 1 - h1.distance(h2) / 64.0

def walk_ast(node):
    yield node.__class__.__name__
    for child in node.children:
        if isinstance(child, list):
            for g in child:
                if isinstance(g, javalang.ast.Node):
                    yield f'{node.__class__.__name__}->{g.__class__.__name__}'
                    yield from walk_ast(g)
        elif isinstance(child, javalang.ast.Node):
            yield f'{node.__class__.__name__}->{child.__class__.__name__}'
            yield from walk_ast(child)

def simhash_ast(src:str)->Simhash:
    ast   = javalang.parse.parse(src)
    feats = list(walk_ast(ast))
    return Simhash(feats)

def ast_similarity(code_a:str, code_b:str)->float:
    h1, h2 = simhash_ast(code_a), simhash_ast(code_b)
    return 1 - h1.distance(h2) / 64.0

# Funciones para crear cadena de Markov
def iter_children(node):
    for ch in node.children:
        if isinstance(ch, jt.Node):
            yield ch
        elif isinstance(ch, (list, tuple)):
            for g in ch:
                if isinstance(g, jt.Node):
                    yield g

def ast_to_seq(node):
    
    seq, stack = [], [node]
    while stack:
        n = stack.pop()
        seq.append(type(n).__name__)        
        stack.extend(reversed(list(iter_children(n))))
    return seq

def build_vocab(ast_objs):
    vocab = set()
    for ast in ast_objs:
        vocab.update(ast_to_seq(ast))
    return {tok: i for i, tok in enumerate(sorted(vocab))}

def transition_matrix(seq, vocab):
    n  = len(vocab)
    M  = [[0]*n for _ in range(n)]
    idx = [vocab[t] for t in seq]
    for i in range(len(idx)-1):
        M[idx[i]][idx[i+1]] += 1
    
    for r in range(n):
        s = sum(M[r])
        if s: M[r] = [x/s for x in M[r]]
    return M          

SPACE = " "

def mat_to_str(mat: np.ndarray) -> str:
    
    return SPACE.join(f"{x:.6f}" for x in mat.ravel())


from functools import lru_cache

@lru_cache(maxsize=None)
def parse_ast(path:str):
    code = Path(path).read_text(encoding='utf-8', errors='ignore')
    return javalang.parse.parse(code)

all_asts = []
for info in tqdm(pairs_dict.values(), total=len(pairs_dict),
                 desc="Parseando AST (vocab)"):
     # codigo 1
    try:
        all_asts.append(parse_ast(info["code1"]))
    except Exception as e:
        print(f"  AST skip {info['code1']}  ({e})")

    # codigo 2
    try:
        all_asts.append(parse_ast(info["code2"]))
    except Exception as e:
        print(f"  AST skip {info['code2']}  ({e})")

VOCAB = build_vocab(all_asts)

Parseando AST (vocab):  83%|████████▎ | 2557/3078 [00:23<00:05, 94.06it/s] 

⚠️  AST skip conplag_version_2/versions\version_2\3d06b643_db7f80a5\db7f80a5.java  ()
⚠️  AST skip conplag_version_2/versions\version_2\3d06b643_db7f80a5\db7f80a5.java  ()


Parseando AST (vocab): 100%|██████████| 3078/3078 [00:27<00:00, 111.75it/s]


In [ ]:
rows = []
total = len(pairs_dict)

for pid, info in tqdm(pairs_dict.items(),
                      total=total,
                      desc="Procesando pares"):
    try:
        # Codigo y ast
        code_a = read_code(info["code1"])
        code_b = read_code(info["code2"])
        ast_a  = parse_ast(info["code1"])
        ast_b  = parse_ast(info["code2"])

        # Similitudes
        simhash_sim = text_similarity(code_a, code_b)
        sim_ast = ast_similarity(code_a, code_b)

        # Markov
        # flat y serializacion para CSV
        M1 = transition_matrix(ast_to_seq(ast_a), VOCAB)
        M2 = transition_matrix(ast_to_seq(ast_b), VOCAB)
        M1 = np.asarray(M1, dtype=np.float32)
        M2 = np.asarray(M2, dtype=np.float32)

        rows.append({
            "id"           : pid,
            "idcode1"      : info["idcode1"],
            "idcode2"      : info["idcode2"],
            "code1"        : info["code1"],   # ruta
            "code2"        : info["code2"],   # ruta
            "simhash"      : simhash_sim,
            "astsimilarity": sim_ast,
            "markov1"      : mat_to_str(M1),
            "markov2"      : mat_to_str(M2),
            "dataset"      : info["dataset"],
            "result"       : info["result"],
        })

    except Exception as e:
        # Cualquier fallo de parseo/similaridad ⇒ se omite el par
        print(f"  {pid} omitido ({e})")

# Guarda el CSV
import pandas as pd
pd.DataFrame(rows).to_csv("dataset.csv", index=False)
print("  dataset.csv generado con", len(rows), "filas")


Procesando pares:  83%|████████▎ | 2553/3078 [00:24<00:05, 100.72it/s]

⚠️  3d06b643_db7f80a5 omitido ()
⚠️  db7f80a5_3d06b643 omitido ()


Procesando pares: 100%|██████████| 3078/3078 [00:32<00:00, 96.00it/s] 


✅  dataset.csv generado con 3076 filas
